In [8]:
!git clone -b CatVTON https://github.com/Harsh-Kesharwani/stable-diffusion.git

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 64 (delta 21), reused 56 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 4.94 MiB | 4.07 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [9]:
cd stable-diffusion/

/kaggle/working/stable-diffusion/stable-diffusion


In [10]:
!git pull

Already up to date.


In [11]:
!wget https://huggingface.co/sd-legacy/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt

--2025-06-11 17:24:50--  https://huggingface.co/sd-legacy/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt
Resolving huggingface.co (huggingface.co)... 3.169.137.119, 3.169.137.111, 3.169.137.5, ...
Connecting to huggingface.co (huggingface.co)|3.169.137.119|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /stable-diffusion-v1-5/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt [following]
--2025-06-11 17:24:51--  https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/f6/56/f656f0fa3b8a40ac76d297fa2a4b00f981e8eb1261963460764e7dd3b35ec97f/c6bbc15e3224e6973459ba78de4998b80b50112b0ae5b5c67113d56b4e366b19?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sd-v1-5-inpainting.ckpt%3B+filename%3D%22sd-v1-5-inpainting.c

In [12]:
!ls

attention.py  encoder.py	  pipeline.py		   test.ipynb
clip.py       interface.py	  README.md		   training.ipynb
ddpm.py       merges.txt	  requirements.txt	   utils.py
decoder.py    model_converter.py  sample_dataset	   VITON_Dataset.py
diffusion.py  model.py		  sd-v1-5-inpainting.ckpt  vocab.json


In [ ]:
# !pip uninstall gdown -y

Found existing installation: gdown 5.2.0
Uninstalling gdown-5.2.0:
  Successfully uninstalled gdown-5.2.0


In [ ]:
# !pip install -U --no-cache-dir gdown --pre

In [ ]:
# !gdown --id 1tLx8LRp-sxDp0EcYmYoV_vXdSc-jJ79w


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1tLx8LRp-sxDp0EcYmYoV_vXdSc-jJ79w

but Gdown can't. Please check connections and permissions.


In [ ]:
# !mkdir data
# !mv test data
# !mv train data
# !mv test_pairs.txt data
# !mv train_pairs.txt data

mkdir: cannot create directory ‘data’: File exists


In [56]:
!ls

agnostic_mask.png  diffusion.py  merges.txt	     requirements.txt
attention.py	   dog.jpg	 model_converter.py  sd-v1-5-inpainting.ckpt
clip.py		   encoder.py	 model.py	     test.ipynb
data		   garment.jpg	 person.jpg	     vocab.json
ddpm.py		   image.png	 pipeline.py	     zalando-hd-resized.zip
decoder.py	   interface.py  README.md


In [57]:
# cat data/train

In [59]:
# !cat data/train_pairs.txt

In [13]:
!mkdir output
!mkdir checkpoints

In [ ]:
!pip install diffusers

In [139]:
import torch
import gc

# Delete all tensors and force garbage collection
torch.cuda.empty_cache()           # Clears unused memory
gc.collect()                       # Python garbage collection

# If you want to delete specific variables:
for obj in dir():
    if 'cuda' in str(locals()[obj]):
        del locals()[obj]
gc.collect()
torch.cuda.empty_cache()


In [140]:
import torch
import gc

torch.cuda.empty_cache()  # Release unused GPU memory
gc.collect()              # Run Python garbage collector

KeyError: '_oh'

In [141]:
import torch
import gc

# Clear CUDA cache and collect garbage
torch.cuda.empty_cache()
gc.collect()

# Delete all user-defined variables except for built-ins and modules
for var in list(globals()):
    if not var.startswith("__") and var not in ["torch", "gc"]:
        del globals()[var]

gc.collect()
torch.cuda.empty_cache()

In [142]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [143]:
import torch

if torch.cuda.is_available():
    used = torch.cuda.memory_allocated() / 1024 ** 2  # in MB
    total = torch.cuda.get_device_properties(0).total_memory / 1024 ** 2  # in MB
    print(f"GPU memory used: {used:.2f} MB / {total:.2f} MB")
else:
    print("CUDA is not available.")

GPU memory used: 17.12 MB / 15095.06 MB


In [144]:
import psutil

mem = psutil.virtual_memory()
total_ram = mem.total / (1024 ** 3)  # in GB
available_ram = mem.available / (1024 ** 3)  # in GB
print(f"Total RAM: {total_ram:.2f} GB")
print(f"Available RAM: {available_ram:.2f} GB")

Total RAM: 31.35 GB
Available RAM: 23.97 GB


In [ ]:
import os
import json
import random
import argparse
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

import numpy as np
from PIL import Image
from tqdm import tqdm
from VITON_Dataset import VITONHDTestDataset

# Import your custom modules
from model import preload_models_from_standard_weights
from ddpm import DDPMSampler  # Fixed import
from utils import check_inputs, compute_vae_encodings, get_time_embedding, prepare_image, prepare_mask_image
from diffusers.utils.torch_utils import randn_tensor

class CatVTONTrainer:
    """CatVTON Training Class with PEFT, CFG and DREAM support"""
    
    def __init__(
        self,
        models: Dict[str, nn.Module],
        train_dataloader: DataLoader,
        val_dataloader: Optional[DataLoader] = None,
        device: str = "cuda",
        learning_rate: float = 1e-5,  # Updated to paper value
        num_epochs: int = 100,
        save_steps: int = 1000,
        output_dir: str = "./checkpoints",
        cfg_dropout_prob: float = 0.1,
        guidance_scale: float = 2.5,
        num_inference_steps: int = 50,
        gradient_accumulation_steps: int = 1,
        max_grad_norm: float = 1.0,
        use_peft: bool = True,
        dream_lambda: float = 10.0,  # DREAM parameter
        resume_from_checkpoint: Optional[str] = None,
        use_mixed_precision: bool = True,  # For memory optimization
        height=512,
        width=512,
    ):
        self.training = True
        self.models = models
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.device = device
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.save_steps = save_steps
        self.output_dir = Path(output_dir)
        self.cfg_dropout_prob = cfg_dropout_prob
        self.guidance_scale = guidance_scale
        self.num_inference_steps = num_inference_steps
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.max_grad_norm = max_grad_norm
        self.use_peft = use_peft
        self.dream_lambda = dream_lambda
        self.use_mixed_precision = use_mixed_precision
        self.height=height
        self.width=width

        self.encoder=self.models.get('encoder', None)
        self.decoder=self.models.get('decoder', None)
        self.diffusion=self.models.get('diffusion', None)

        self.generator = torch.Generator(device=device)
        
        # Create output directory
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # Setup mixed precision training
        if self.use_mixed_precision:
            self.scaler = torch.cuda.amp.GradScaler()

        self.weight_dtype = torch.float16 if use_mixed_precision else torch.float32
        
        # Setup models and optimizers
        self._setup_training()
        
        # Initialize scheduler and sampler
        self.scheduler = DDPMSampler(self.generator, num_training_steps=1000)
        
        # Resume from checkpoint if provided
        self.global_step = 0
        self.current_epoch = 0
        if resume_from_checkpoint:
            self._load_checkpoint(resume_from_checkpoint)
    
    def _setup_training(self):
        """Setup models for training with PEFT"""
        # Move models to device with mixed precision
        for name, model in self.models.items():
            model.to(self.device)
            if self.use_mixed_precision and name != 'encoder':  # Keep encoder in float32 for stability
                model.half()
        
        # Freeze all parameters first
        for model in self.models.values():
            for param in model.parameters():
                param.requires_grad = False
        
        # Enable training for specific layers based on PEFT strategy
        if self.use_peft:
            self._enable_peft_training()
        else:
            # Enable full training for diffusion model
            for param in self.models['diffusion'].parameters():
                param.requires_grad = True
        
        # Collect trainable parameters
        trainable_params = []
        total_params = 0
        trainable_count = 0
        
        for name, model in self.models.items():
            for param_name, param in model.named_parameters():
                total_params += param.numel()
                if param.requires_grad:
                    trainable_params.append(param)
                    trainable_count += param.numel()
        
        print(f"Total parameters: {total_params:,}")
        print(f"Trainable parameters: {trainable_count:,} ({trainable_count/total_params*100:.2f}%)")
        
        # Verify we're close to the paper's 49.57M parameters for self-attention only
        if self.use_peft:
            expected_params = 49_570_000  # 49.57M
            if abs(trainable_count - expected_params) > 5_000_000:  # 5M tolerance
                print(f"Warning: Expected ~{expected_params:,} trainable parameters, got {trainable_count:,}")
        
        # Setup optimizer - AdamW as per paper
        self.optimizer = AdamW(
            trainable_params,
            lr=self.learning_rate,
            betas=(0.9, 0.999),
            weight_decay=1e-2,
            eps=1e-8
        )
        
        # Setup learning rate scheduler (constant as per paper)
        # For constant LR, we can use a dummy scheduler
        self.lr_scheduler = torch.optim.lr_scheduler.LambdaLR(
            self.optimizer, lr_lambda=lambda epoch: 1.0
        )
    
    def _enable_peft_training(self):
        """Enable PEFT training - only self-attention layers (49.57M parameters)"""
        print("Enabling PEFT training (self-attention layers only)")
        
        unet = self.diffusion.unet
        
        # Enable attention layers in encoders
        for layers in [unet.encoders, unet.decoders]:
            for layer in layers:
                if hasattr(layer, 'attention'):  # UNET_AttentionBlock
                    for param in layer.attention.parameters():
                        param.requires_grad = True
                elif hasattr(layer, 'attention_1'):  # Alternative naming
                    for param in layer.attention_1.parameters():
                        param.requires_grad = True
        
        # Enable attention layers in bottleneck
        for layer in unet.bottleneck:
            if hasattr(layer, 'attention'):
                for param in layer.attention.parameters():
                    param.requires_grad = True
            elif hasattr(layer, 'attention_1'):
                for param in layer.attention_1.parameters():
                    param.requires_grad = True
    
    def _apply_cfg_dropout(self, garment_latent: torch.Tensor) -> torch.Tensor:
        """Apply classifier-free guidance dropout (10% chance)"""
        if self.training and random.random() < self.cfg_dropout_prob:
            # Replace with zero tensor for unconditional generation
            return torch.zeros_like(garment_latent)
        return garment_latent
    
    def compute_loss(self, batch: Dict[str, torch.Tensor]) -> torch.Tensor:
        """Compute MSE loss for denoising with DREAM strategy"""
        person_images = batch['person'].to(self.device)
        cloth_images = batch['cloth'].to(self.device)
        masks = batch['mask'].to(self.device)
        
        batch_size = person_images.shape[0]

        concat_dim = -2  # FIXME: y axis concat
        # Prepare inputs to Tensor
        image, condition_image, mask = check_inputs(person_images, cloth_images, masks, self.width, self.height)
        image = prepare_image(person_images).to(self.device, dtype=self.weight_dtype)
        condition_image = prepare_image(cloth_images).to(self.device, dtype=self.weight_dtype)
        mask = prepare_mask_image(masks).to(self.device, dtype=self.weight_dtype)
        # Mask image
        masked_image = image * (mask < 0.5)

        with torch.cuda.amp.autocast(enabled=self.use_mixed_precision):
        # VAE encoding
            masked_latent = compute_vae_encodings(masked_image, self.encoder)
            condition_latent = compute_vae_encodings(condition_image, self.encoder)
            mask_latent = torch.nn.functional.interpolate(mask, size=masked_latent.shape[-2:], mode="nearest")
            del image, mask, condition_image


            # Apply CFG dropout to garment latent
            condition_latent = self._apply_cfg_dropout(condition_latent)
            
            # Concatenate latents
            masked_latent_concat = torch.cat([masked_latent, condition_latent], dim=concat_dim)
            mask_latent_concat = torch.cat([mask_latent, torch.zeros_like(mask_latent)], dim=concat_dim)

            target_latents = masked_latent_concat

            noise=randn_tensor(
                masked_latent_concat.shape,
                generator=self.generator,
                device=masked_latent_concat.device,
                dtype=self.weight_dtype,
            )

            timesteps = torch.randint(1, 1000, size=(1,))[0].long().item()

            timesteps_embedding=get_time_embedding(timesteps)

            # Add noise to latents
            noisy_latents = self.scheduler.add_noise(target_latents, timesteps, noise)

            non_inpainting_latent_model_input = noisy_latents
            inpainting_latent_model_input = torch.cat([
                non_inpainting_latent_model_input, 
                mask_latent_concat, 
                masked_latent_concat
            ], dim=1).to(self.device, dtype=self.weight_dtype)

            # DREAM strategy implementation
            if self.dream_lambda > 0:
            
                # print(f"Model input shape: {model_input.shape}")
                # print(f"Time embeddings shape: {time_embeddings.shape}")
                
                # Get initial noise prediction
                with torch.no_grad():
                    epsilon_theta = self.diffusion(
                    inpainting_latent_model_input,
                    timesteps_embedding
                )

                # print(f"Predicted noise shape: {epsilon_theta.shape}")
                
                # Apply DREAM: zˆt = √αt*z0 + √(1-αt)*(ε + λ*εθ)
                alphas_cumprod = self.scheduler.alphas_cumprod.to(device=self.device, dtype=self.weight_dtype)
                sqrt_alpha_prod = alphas_cumprod[timesteps] ** 0.5
                sqrt_one_minus_alpha_prod = (1 - alphas_cumprod[timesteps]) ** 0.5
                
                # Reshape for broadcasting
                sqrt_alpha_prod = sqrt_alpha_prod.view(-1, 1, 1, 1)
                sqrt_one_minus_alpha_prod = sqrt_one_minus_alpha_prod.view(-1, 1, 1, 1)
                
                # DREAM noise combination
                dream_noise = noise + self.dream_lambda * epsilon_theta

                dream_noisy_latents = sqrt_alpha_prod * target_latents + sqrt_one_minus_alpha_prod * dream_noise

                dream_model_input = torch.cat([
                    dream_noisy_latents, 
                    mask_latent_concat, 
                    masked_latent_concat
                ], dim=1)

                predicted_noise= self.diffusion(
                    dream_model_input,
                    timesteps_embedding
                )
                # DREAM loss: |(ε + λεθ) - εθ(ẑt, t)|²
                loss = F.mse_loss(predicted_noise, dream_noise)
            else:
                # Standard training without DREAM
                predicted_noise = self.diffusion(
                    inpainting_latent_model_input,
                    timesteps_embedding,
                )

                # Standard MSE loss
                loss = F.mse_loss(predicted_noise, noise)
            
        return loss
    
    def train_epoch(self) -> float:
        """Train for one epoch"""
        self.models['diffusion'].train()
        total_loss = 0.0
        num_batches = len(self.train_dataloader)
        
        progress_bar = tqdm(self.train_dataloader, desc=f"Epoch {self.current_epoch+1}")
        
        for step, batch in enumerate(progress_bar):
            # Compute loss with mixed precision
            if self.use_mixed_precision:
                with torch.cuda.amp.autocast():
                    loss = self.compute_loss(batch)
                
                # Scale loss for gradient accumulation
                loss = loss / self.gradient_accumulation_steps
                
                # Backward pass with scaling
                self.scaler.scale(loss).backward()
            else:
                loss = self.compute_loss(batch)
                loss = loss / self.gradient_accumulation_steps
                loss.backward()
            
            # Gradient accumulation
            if (step + 1) % self.gradient_accumulation_steps == 0:
                if self.use_mixed_precision:
                    # Unscale gradients and clip
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(
                        [p for p in self.models['diffusion'].parameters() if p.requires_grad],
                        self.max_grad_norm
                    )
                    
                    # Optimizer step with scaling
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                else:
                    # Clip gradients
                    torch.nn.utils.clip_grad_norm_(
                        [p for p in self.models['diffusion'].parameters() if p.requires_grad],
                        self.max_grad_norm
                    )
                    self.optimizer.step()
                
                self.lr_scheduler.step()
                self.optimizer.zero_grad()
                self.global_step += 1
            
            total_loss += loss.item() * self.gradient_accumulation_steps
            
            # Update progress bar
            progress_bar.set_postfix({
                'loss': loss.item() * self.gradient_accumulation_steps,
                'lr': self.optimizer.param_groups[0]['lr'],
                'step': self.global_step
            })
            
            # Save checkpoint
            if self.global_step % self.save_steps == 0:
                self._save_checkpoint()
            
            # Clear cache periodically to prevent OOM
            if step % 50 == 0:
                torch.cuda.empty_cache()
        
        return total_loss / num_batches
    
    def train(self):
        """Main training loop"""
        print(f"Starting training for {self.num_epochs} epochs")
        print(f"Total training steps: {self.num_epochs * len(self.train_dataloader)}")
        print(f"Using DREAM with lambda = {self.dream_lambda}")
        print(f"Mixed precision: {self.use_mixed_precision}")
        
        
        for epoch in range(self.current_epoch, self.num_epochs):
            self.current_epoch = epoch
            
            # Train
            train_loss = self.train_epoch()
            
            print(f"Epoch {epoch+1}/{self.num_epochs}")
            print(f"Train Loss: {train_loss:.6f}")
            
            # Save epoch checkpoint
            if (epoch + 1) % 10 == 0:
                self._save_checkpoint(epoch_checkpoint=True)
            
            # Clear cache at end of epoch
            torch.cuda.empty_cache()
    
    def _save_checkpoint(self, is_best: bool = False, epoch_checkpoint: bool = False):
        """Save model checkpoint"""
        checkpoint = {
            'global_step': self.global_step,
            'current_epoch': self.current_epoch,
            'diffusion_state_dict': self.models['diffusion'].state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'lr_scheduler_state_dict': self.lr_scheduler.state_dict(),
            'dream_lambda': self.dream_lambda,
            'use_peft': self.use_peft,
        }
        
        if self.use_mixed_precision:
            checkpoint['scaler_state_dict'] = self.scaler.state_dict()
        
        if is_best:
            checkpoint_path = self.output_dir / "best_model.pth"
        elif epoch_checkpoint:
            checkpoint_path = self.output_dir / f"checkpoint_epoch_{self.current_epoch+1}.pth"
        else:
            checkpoint_path = self.output_dir / f"checkpoint_step_{self.global_step}.pth"
        
        torch.save(checkpoint, checkpoint_path)
        print(f"Checkpoint saved: {checkpoint_path}")
    
    def _load_checkpoint(self, checkpoint_path: str):
        """Load model checkpoint"""
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        
        self.global_step = checkpoint['global_step']
        self.current_epoch = checkpoint['current_epoch']
        self.dream_lambda = checkpoint.get('dream_lambda', 10.0)
        
        self.models['diffusion'].load_state_dict(checkpoint['diffusion_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.lr_scheduler.load_state_dict(checkpoint['lr_scheduler_state_dict'])
        
        if self.use_mixed_precision and 'scaler_state_dict' in checkpoint:
            self.scaler.load_state_dict(checkpoint['scaler_state_dict'])
        
        print(f"Checkpoint loaded: {checkpoint_path}")
        print(f"Resuming from epoch {self.current_epoch}, step {self.global_step}")


def create_dataloaders(args) -> Tuple[DataLoader, Optional[DataLoader]]:
    """Create training and validation dataloaders"""
    # Dataset
    if args.dataset_name == "vitonhd":
        dataset = VITONHDTestDataset(args)
    else:
        raise ValueError(f"Invalid dataset name {args.dataset}.")
    print(f"Dataset {args.dataset_name} loaded, total {len(dataset)} pairs.")
    dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=args.dataloader_num_workers
    )
    
    return dataloader


def main():
    args=argparse.Namespace()
    args.__dict__= {
        "base_model_path": "inkpunk-diffusion-v1.ckpt",
        "resume_path": "zhengchong/CatVTON",
        "dataset_name": "vitonhd",
        "data_root_path": "/kaggle/input/viton-hd-dataset",
        "output_dir": "./output",
        "seed": 42,
        "batch_size": 2,
        "num_inference_steps": 50,
        "guidance_scale": 2.5,
        "width": 384,
        "height": 512,
        "repaint": True,
        "eval_pair": False,
        "concat_eval_results": True,
        "allow_tf32": True,
        "dataloader_num_workers": 4,
        "mixed_precision": 'no',
        "concat_axis": 'y',
        "enable_condition_noise": True,
        "device":"cuda",
        "num_training_steps": 16000,
        "learning_rate": 1e-5,
        "gradient_accumulation_steps": 128,  # Simulate batch size 128
        "max_grad_norm": 1.0,
        "use_peft": True,
        "cfg_dropout_prob": 0.1,
        "dream_lambda": 10.0,
        "use_mixed_precision": True,
        "output_dir": "./checkpoints",
        "save_steps": 1000,
        "resume_from_checkpoint": None,
        "is_train": True
    }
    
    # Calculate epochs from training steps
    # This will be calculated after dataloader creation
    
    # Set random seeds
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    random.seed(args.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)
    
    # Optimize CUDA settings for memory
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True  
    torch.set_float32_matmul_precision("high")

    # Load pretrained models
    print("Loading pretrained models...")
    models = preload_models_from_standard_weights(args.base_model_path, args.device)
    
    # Create dataloaders
    print("Creating dataloaders...")
    train_dataloader = create_dataloaders(args)
    
    # Calculate epochs from training steps
    steps_per_epoch = len(train_dataloader) // args.gradient_accumulation_steps
    num_epochs = (args.num_training_steps + steps_per_epoch - 1) // steps_per_epoch
    print(f"Training for {num_epochs} epochs ({args.num_training_steps} steps)")
    args.num_epochs = num_epochs
    print(f"Steps per epoch: {steps_per_epoch}")
    print(f"Total training steps: {args.num_training_steps}")
    print(f"Total epochs: {num_epochs}")
    # Initialize trainer
    print("Initializing trainer...")    
    trainer = CatVTONTrainer(
        models=models,
        train_dataloader=train_dataloader,
        device=args.device,
        learning_rate=args.learning_rate,
        num_epochs=args.num_epochs,
        save_steps=args.save_steps,
        output_dir=args.output_dir,
        cfg_dropout_prob=args.cfg_dropout_prob,
        guidance_scale=args.guidance_scale,
        num_inference_steps=50,  # Fixed as per paper
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        max_grad_norm=args.max_grad_norm,
        use_peft=args.use_peft,
        dream_lambda=args.dream_lambda,
        resume_from_checkpoint=args.resume_from_checkpoint,
        use_mixed_precision=args.use_mixed_precision
    )
    # Start training
    print("Starting training...")
    trainer.train() 

if __name__ == "__main__":
    main()


2025-06-11 17:26:05.199950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749662765.402784      71 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749662765.463921      71 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


KeyboardInterrupt: 